In [33]:
import pandas as pd
from bs4 import BeautifulSoup as bs  
import requests
from splinter import Browser
import pymongo
import selenium

from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from pymongo import MongoClient

In [34]:
executable_path = {'executable_path': 'C:\\Users\\nmans\\chromedriver_win32\\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [35]:
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)
html = browser.html
soup = bs(html, 'html.parser')

### NASA Mars News

In [43]:
#collect the latest News Title and Paragraph Text
news=soup.find_all('ul', class_='item_list ')
soup = bs(html, 'html.parser')

news_list=[{'title':last_news.find('div', 'class_=content_title'),
            'paragragh':last_news.find('div', class_='article_teaser_body')
           } 
           for last_news in news]

for news in news_list:
    print(news.prettify())

### JPL Mars Space Images - Featured Image

In [44]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image

featured_image_url="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
soup = bs(html, 'html.parser')

# Make sure to find the image url to the full size

browser.visit(featured_image_url)
main_img_fig=browser.find_by_id('full_image') 
main_img_fig.click()


### Mars weather 

In [62]:
# Save the tweet text for the weather report 

mars_weather_url="https://twitter.com/marswxreport?lang=en"
browser.visit(featured_image_url)
soup=bs(browser.html,'html.parser')

mars_weather=soup.find('a',class_='css-4rbku5 css-18t94o4 css-901oao css-16my406 r-13gxpu9 r-1loqt21 r-4qtqp9 r-1qd0xha r-ad9z0x r-zso239 r-bcqeeo r-qvutc0')

mars_weather=soup.find('div',class_='wysiwyg_content')
print(mars_weather)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=79.0.3945.88)


### Mars Facts

In [40]:
# use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
mars_fact_url="https://space-facts.com/mars/"
browser.visit(mars_fact_url)
soup = bs(html, 'html.parser')

table=pd.read_html(mars_fact_url)
planet_fact=table[1]
planet_fact.columns=['Description','Mars','Earth']
planet_fact.set_index('Description', inplace=True)
planet_fact


,Mars,Earth
Description,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-153 to 20 °C,-88 to 58°C


In [26]:
planet_fact.to_html('html')

### Mars Hemispheres

In [63]:
# Obtain high resolution images for each of Mar's hemispheres
hemisphere_url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(mars_fact_url)
soup = bs(html, 'html.parser')
hemisphere_image_urls=[]


# hemisphere_image_urls=[
#     {"title": "Cerberus Hemisphere Enhanced", "img_url": "https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced",
#      "title": "Schiaparelli Hemisphere Enhanced","img_url":"https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced",
#      "title": "Syrtis Major Hemisphere Enhanced", "img_url":"https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced",
#      "title": "Valles Marineris Hemisphere Enhanced", "img_url":"https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced"}
        
# ]



# Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name

# Append the dictionary with the image url string and the hemisphere title to a list

### MongoDB and Flask Application

In [61]:
#Use MongoDB with Flask templating to create a new HTML page that displays all of the information that was scraped from the URLs above

# Use flask_pymongo to set up mongo connection
# app.config["MONGO_URI"] = "mongodb://localhost:27017/craigslist_app"
# mongo = PyMongo(app)
db_url = "mongodb://localhost:27017"
client = MongoClient(db_url)
db = client.mars_mission_db

@app.route("/")
def()index():
      mars_mission = db.mars_mission.find_one()
    return render_template("index.html", mars_mission=mars_mission)

@app.route("/scrape")
def scraper():
    mars_mission = db.mars_mission
    mars_mission_data = scrape_mission_to_mars.scrape()
    mars_mission.update({}, mars_mission_data, upsert=True)
    return 'Mission accomplished'


if __name__ == "__main__":
    app.run(debug=True)